In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.animation as animation

class Pendulum:
    def __init__(self, theta1, theta2, dt):
        self.theta1 = theta1
        self.theta2 = theta2
        self.p1 = 0.0
        self.p2 = 0.0
        self.dt = dt
        self.g = 9.81
        self.trajectory = [self.polar_to_cartesian()]

    def polar_to_cartesian(self):
        x1 = np.sin(self.theta1)
        x2 = x1 + np.sin(self.theta2)
        y1 = -np.cos(self.theta1)
        y2 = y1 - np.cos(self.theta2)
        return np.array([[0.0, 0.0], [x1, y1], [x2, y2]])

    def work(self):
        # Corrected equations
        self.theta1 += ((self.p1 - (self.p2 * np.cos(self.theta1 - self.theta2))) /
                       (1 + (np.sin(self.theta1 - self.theta2)) ** 2)) * self.dt
        self.theta2 += ((2 * self.p2 - self.p1 * np.cos(self.theta1 - self.theta2)) /
                       (1 + (np.sin(self.theta1 - self.theta2)) ** 2)) * self.dt
        self.p1 += (-2 * self.g * np.sin(self.theta1) -
                    ((self.p1 * self.p2 * np.sin(self.theta1 - self.theta2)) /
                     (1 + (np.sin(self.theta1 - self.theta2)) ** 2)) +
                    (((self.p1 ** 2 + 2 * self.p2 ** 2) *
                      (self.p1 * self.p2 * np.cos(self.theta1 - self.theta2))) /
                     (2 * (1 + (np.sin(self.theta1 - self.theta2)) ** 2) ** 2) *
                     np.sin(2 * (self.theta1 - self.theta2)))) * self.dt
        self.p2 += (-self.g * np.sin(self.theta2) +
                    ((self.p1 * self.p2 * np.sin(self.theta1 - self.theta2)) /
                     (1 + (np.sin(self.theta1 - self.theta2)) ** 2)) -
                    (((self.p1 ** 2 + 2 * self.p2 ** 2) *
                      (self.p1 * self.p2 * np.cos(self.theta1 - self.theta2))) /
                     (2 * (1 + (np.sin(self.theta1 - self.theta2)) ** 2) ** 2) *
                     np.sin(2 * (self.theta1 - self.theta2)))) * self.dt

        new_position = self.polar_to_cartesian()
        self.trajectory.append(new_position)
        return new_position


class Animator:
    def __init__(self, pendulum, draw_trace=False):
        self.pendulum = pendulum
        self.draw_trace = draw_trace
        self.time = 0.0
        self.fig, self.ax = plt.subplots()
        self.ax.set_ylim(-2.5, 2.5)  # Corrected method name
        self.ax.set_xlim(-2.5, 2.5)  # Corrected method name
        self.time_text = self.ax.text(0.05, 0.95, '', horizontalalignment='left', verticalalignment='top',
                                      transform=self.ax.transAxes)
        self.line, = self.ax.plot(self.pendulum.trajectory[-1][:, 0], self.pendulum.trajectory[-1][:, 1], marker='o')

        if self.draw_trace:
            self.trace, = self.ax.plot([a[2, 0] for a in self.pendulum.trajectory],
                                       [a[2, 1] for a in self.pendulum.trajectory])  # Corrected method name

    def advance_time_step(self):
        while True:
            self.time += self.pendulum.dt
            yield self.pendulum.work()

    def update(self, data):
        self.time_text.set_text('Elapsed time: {:6.2f} s'.format(self.time))
        self.line.set_ydata(data[:, 1])
        self.line.set_xdata(data[:, 0])
    

        if self.draw_trace:
            self.trace.set_xdata([a[2, 0] for a in self.pendulum.trajectory])
            self.trace.set_ydata([a[2, 1] for a in self.pendulum.trajectory])
        return self.line,

    def animate(self):
        self.animator = animation.FuncAnimation(self.fig, self.update, self.advance_time_step, interval=25, blit=False)


pendulum = Pendulum(theta1=np.pi, theta2=np.pi - 0.01, dt=0.01)

animator = Animator(pendulum=pendulum, draw_trace=True)
animator.animate()
plt.show()